#Grupo 9 Práctica 2

Notebook de la práctica 2 realizado por Alejandro García Berrocal (100451059) y Lucas Gallego Bravo (100429005)
GitHub: https://github.com/AlejandroGB-9/Grupo9-Practica2

#Prerequisitos:

##Librerías e imports necesarios

In [ ]:
!pip3 install numpy
!pip3 install pandas
!pip3 install scikit-learn
!pip3 install scikit-optimize
!pip3 install skopt
!pip3 install matplotlib
!pip3 install seaborn
!pip3 install statsmodels
!pip3 install lightgbm

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import time
from scipy.stats import loguniform, randint, uniform
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler
from sklearn.model_selection import GridSearchCV, StratifiedKFold, RandomizedSearchCV
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, chi2
from skopt import BayesSearchCV
from skopt.space import Integer
from sklearn.pipeline import Pipeline
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score, balanced_accuracy_score, confusion_matrix, make_scorer
from sklearn.impute import SimpleImputer
from skopt.space import Integer, Real
from sklearn.model_selection import train_test_split

##Extracción de datos

In [ ]:
datos = pd.read_pickle('./attrition_available_9.pkl')

#EDA

En este EDA se han analizado los datos provenientes del archivo attrition_available_9.pkl, el cual contiene los datos de los empleados de una empresa. En este EDA se han analizado los datos de los empleados que han abandonado la empresa, es decir, aquellos que tienen el valor 1 en la columna Attrition. Aparte de realizar ese estudio se ha realizado un estudio de la correlación entre las variables numéricas y se ha realizado un estudio de los datos faltantes.

In [ ]:
#------------------------------------------
#Cuantos datos faltantes o NaN hay

print(datos.head())
print()
print("Información del dataset:\n ", datos.info())
print()
print(datos['Attrition'].value_counts(normalize=True))
print()
print("Suma de datos que son nulos o NA:\n", datos.isnull().sum())
print()
print(f"Cuantos valores NaN|missing values contiene el target: {datos['Attrition'].isnull().sum()}")
print()
print(f"Tiene el mismo valor en todas las filas el atributo Over18: {datos['Over18'].value_counts()/len(datos)}")
print()

#------------------------------------------
#Obtener los datos numéricos y categoricos

cols_cat = datos.select_dtypes(include=['object']).columns
cols_num = datos.select_dtypes(include=['int64', 'float64']).columns

print(f"Los valores categoricos son: {cols_cat}")
print()
print(f"Los valores numericos son: {cols_num}")
    
#------------------------------------------
#Estudio de la correlación entre las variables numéricas

X = datos.select_dtypes(include=['int64', 'float64'])
df = pd.DataFrame(X)
print()
print(df)

corr_matrix = df.corr().abs()
print()
print(corr_matrix)

f, ax = plt.subplots(figsize=(12, 9))
plt.title('Correlación entre variables', y=1, size=16)
sns.heatmap(corr_matrix, vmax=.8, square=True)

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
print()
print(upper)

to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
print()
print("Estos son los atributos a eliminar:\n")
print(to_drop)

#------------------------------------------
#Drop the los valores que no se van a utilizar o no pertenecen al dataset
cols_cat = cols_cat.drop('Over18')
cols_cat = cols_cat.drop('Attrition')
cols_num = cols_num.drop('EmployeeCount')
cols_num = cols_num.drop('StandardHours')
cols_num = cols_num.drop('EmployeeID')

col_cat = []
col_num = []
for i in cols_cat:
    col_cat.append(str(i))
for i in cols_num:
    col_num.append(str(i))

# Define the transformers
numerical_transformer = SimpleImputer(strategy='mean')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])


# Define the preprocessor
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, col_num),
    ('cat', categorical_transformer, col_cat),
    
])

print()
print("Preprocesador para usar en los modelos debido a la cantidad de missing values | NaN:\n")
print(preprocessor)

Sabemos que se trata de un problema de clasificación desbalanceado, por lo que se debe tener en cuenta a la hora de evaluar los modelos. Además, se debe tener en cuenta que el objetivo es predecir si un empleado se va a ir de la empresa o no por lo que lo haremos con métricas como f1_score, balanced_accuracy y una matrix de confusión. Como conclusión del EDA, los atributos Over18, EmployeeCount, StandardHours son valores constantes por lo que se han de eliminar de los datos, además el atributo EmployeeID no tiene ninguna relevancia para el modelo. Para tratar la falta de datos y otros se ha planteado un preprocesado de datos que se encuentra al final del estudio. Este preproceso realiza una imputación simple sobre los atributos, siendo la media para los numéricos y los más frecuentes para los categóricos. Por otro lado, hemos considerado aplicar un OneHotEncoder a los atributos categóricos para que los modelos puedan trabajar con ellos. De la misma forma se debe aplicar al target por lo que hemos optado por hacer un reemplazo de valores al extraer los conjuntos que se verán ahora. 

#Construcción de modelos

##Extracción del conjunto de datos train/test

In [ ]:
#------------------------------------------
#Se realizará la exrtacción de los datos de entrenamiento y test con el preprocesador obtenido en el EDA
datos = pd.read_pickle('./attrition_available_9.pkl')
target = datos['Attrition']
datos.drop(columns=['Attrition','Over18','EmployeeCount','StandardHours','EmployeeID'], inplace=True)
target = target.replace({'Yes': 1, 'No': 0})
original_columns = pd.get_dummies(datos).columns
data = pd.DataFrame(preprocessor.fit_transform(datos))


#Se reparte el dataset en X y y
ds = data.T
X = [] 
y = target.T.tolist()
for i in ds.columns:
    X.append(ds[i].tolist())

#------------------------------------------
#Obtenemos los conjuntos de entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=9)

##Que se tendrá en cuenta a la hora de evaluar los modelos

Por una parte, se ha de tener en cuenta que se trata de un problema de clasificación desbalanceado, por lo que se debe tener en cuenta a la hora de evaluar los modelos. Además, se debe tener en cuenta que el objetivo es predecir si un empleado se va a ir de la empresa o no por lo que lo haremos con métricas como f1_score, balanced_accuracy y una matrix de confusión. Por lo que estas métricas serán las que se tendrán en cuenta a la hora de evaluar los modelos y por la que se elegirá el mejor modelo. Es importante saber de manera teórica por qué se elige un modelo en base a estas métricas. 

En F1 score, si nuestro resultado es un valor alto cercano a 1 indica que el modelo ha alcanzado un buen equilibrio entre precision y recall, identificando de manera correcta instancias tanto positivas como negativas. Esto se considera como una salida deseada, particularmente para este tipo de problema. Si está por encima de 0.5 se sugiere that que el modelo esta funcionando razonadamente bien. El modelo estaría alcanzando un buen equilibrio entre precision y recall, aunque podría mejorarse. Si está cerca de 0.5 el funcionamiento es mediocre y no estaría funcionando mejor que un random guessing. Si está por debajo de 0.5 el modelo funciona mal y podría estar teniendo problema para clasificar instancias correctamente y podría estar sesgado hacia una clase o sufrir errores significativos. 

En balanced accuracy, si nuestro resultado es un valor alto cercano a 1 indica que el modelo funciona bien en términos de clasificar correctamente las clases mayoritarias y minoritarias. Si está por encima de 0.5 el modelo funciona mejor que un random guessing, es decir, hace predicciones precisas aunque haya alguna diferencia de funcinamiento entre clases. Si está cerca de 0.5 el funcionamiento es mediocre y no estaría funcionando mejor que un random guessing y podría estar sesgado hacia una clase. Si está por debajo de 0.5 el modelo funciona mal y podría estar teniendo problema para clasificar instancias correctamente y podría estar sesgado hacia una clase.

En una matriz de confusión se puede resumir el funcionamiento de un modelo de clasificación mostrando la cuenta de 'true positives' (TP-número de instancias positivas predictas correctamente), 'true negatives' (TN-número de instancias negativas predictas correctamente), 'false positives'(FP-número de instancias positivas predictas incorrectamente) y 'false negatives'(FN-número de instancias negativas predictas incorrectamente). Con estos se puede clacular el accuracy ((TP + TN) / (TP + TN + FP + FN)), precision (TP / (TP + FP)), recall (TP / (TP + FN)) y f1 score (2 * (precision * recall) / (precision + recall)). Con la matriz de confusión se puede tener una visión más clara de cómo funciona el modelo, por lo que se puede analizar las clases con sus fuertes. Con las distribuciones de esta matriz se puede ver si el modelo está sesgado hacia alguna clase o si tiene problemas para clasificar instancias correctamente.

Por otro lado, evaluar los modelos con validaciones cruzadas. En nuestro caso se ha propuesto GridSearchCV, RandomizedSearchCV y BayesSearchCV. Se ha decidido probar el método de Grid Search ya que realiza una búsqueda exhaustiva de todos los posibles valores de hiper parámetros especificados además de evaluar el rendimiento del modelo para cada combinación de estos. Por otro lado, Bayes Search encuentra la combinación de hiper parámetros óptima y puede ser especialmente útil si se trata de ajustar muchos hiper parámetros sin realizar una búsqueda exhaustiva de todas las combinaciones. Finalmente, Random Search consideramos que es uno de los más útiles al no conocer los valores óptimos de los hiper parámetros. Este modelo es perfecto al tener un enfoque más exploratorio realizando una búsqueda aleatoria en un rango de valores de cada hiper parámetro y evalúa el rendimiento de este para cada combinación aleatoria.

Los modelos de Boosting que se han elegido son HistGradientBoosting de scikit learn y LightGBM como librería externa. Se ha escogido HistGradientBoosting por ser bueno debido a su eficiencia computacional, manejo eficiente de datos categóricos, menor consumo de memoria, buen rendimiento predictivo y su compatibilidad con la biblioteca scikit-learn. Por otro lado, se ha elegido LightGBM por que es una biblioteca de gradient boosting conocida por su eficiencia, velocidad y capacidad para manejar conjuntos de datos grandes. Ofrece un rendimiento predictivo superior, flexibilidad para personalizar el modelo y es compatible con varias plataformas. Es una opción popular para problemas de aprendizaje automático que requieren modelos precisos y rápidos.

##Modelo 1: Regresión Logística

In [ ]:
#--------------------------------------
# Logistic Regression
#--------------------------------------

inicio = time.time()
scaler = StandardScaler()
clf = LogisticRegression(random_state=9, class_weight='balanced')
pipe_clf = Pipeline([('scaler', scaler), ('clf', clf)])
pipe_clf.fit(X_train, y_train)
y_pred = pipe_clf.predict(X_test)
f1 = f1_score(y_test, y_pred)
bal_acc = balanced_accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo = final - inicio
print(f"El tiempo tardado es de {tiempo} segundos.\n")
print(f"El f1 es de {f1}\n")
print(f"El balanced accuracy es de {bal_acc}\n")
print(f"La matriz de confusión es: \n{cm}\n")

#--------------------------------------
# Logistic Regression
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with Hyperparameter Tuning
#--------------------------------------

param = { 'clf__class_weight': ['balanced', None]}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = None
b_clf = LogisticRegression(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring='f1', n_jobs=-1).fit(X_train, y_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(X_train, y_train)
final = time.time()
tiempo_bal_acc = final - inicio
inicio = time.time()
y_pred = clf_f1.predict(X_test)
clf_cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_conf_matrix} segundos para confusión matrix.\n")
print(f"El confusion matrix es de: \n{clf_cm}\n")

#--------------------------------------
# Boosting with Hyperparameter Tuning
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with Hyperparameter Tuning
#--------------------------------------

param = {'clf__class_weight': ['balanced', None]}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = None
b_clf = LogisticRegression(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = RandomizedSearchCV(pipe_clf, param_distributions=param, cv=inner, scoring='f1', n_jobs=-1).fit(X_train, y_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = RandomizedSearchCV(pipe_clf, param_distributions=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(X_train, y_train)
final = time.time()
tiempo_bal_acc = final - inicio
inicio = time.time()
y_pred = clf_f1.predict(X_test)
clf_cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_conf_matrix} segundos para confusión matrix.\n")
print(f"El confusion matrix es de: \n{clf_cm}\n")

#--------------------------------------
# Boosting with Hyperparameter Tuning
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with Hyperparameter Tuning
#--------------------------------------

param = {'clf__class_weight': ['balanced', None]}
inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = StandardScaler()
b_clf = LogisticRegression(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = BayesSearchCV(pipe_clf, param, cv=inner, scoring='f1', n_jobs=-1).fit(X_train, y_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = BayesSearchCV(pipe_clf, param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(X_train, y_train)
final = time.time()
tiempo_bal_acc = final - inicio
inicio = time.time()
y_pred = clf_bal_acc.predict(X_test)
clf_cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_conf_matrix} segundos para confusión matrix.\n")
print(f"El confusion matrix es de: \n{clf_cm}\n")

#--------------------------------------
# Boosting with Hyperparameter Tuning
#--------------------------------------

Entre las pruebas y evaluaciones realizadas se ha decidido que el mejor modelo de Regresión Logística es el primero, el cual no incluye ninguna validación cruzada a pesar de que es un modelo malo por sus resultados en las métricas escogidas. 

##Modelo 2: Boosting

##Modelo 2.1: Hist Gradient Boosting

In [ ]:
#--------------------------------------
# Boosting with Hyperparameter Tuning
#--------------------------------------

param = { 'clf__class_weight': ['balanced', None],
         'clf__max_depth': [ 10, 15, 20 ],
         'clf__learning_rate': [10, 1, 0.1]}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = None
b_clf = HistGradientBoostingClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring='f1', n_jobs=-1).fit(X_train, y_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(X_train, y_train)
final = time.time()
tiempo_bal_acc = final - inicio
inicio = time.time()
y_pred = clf_f1.predict(X_test)
clf_cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_conf_matrix} segundos para confusión matrix.\n")
print(f"El confusion matrix es de: \n{clf_cm}\n")

#--------------------------------------
# Boosting with Hyperparameter Tuning
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with Hyperparameter Tuning
#--------------------------------------

param = {'clf__class_weight': ['balanced', None],
         'clf__max_depth': randint(low=10, high=15),
         'clf__learning_rate': uniform( 0.1, 1)}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = RobustScaler()
b_clf = HistGradientBoostingClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = RandomizedSearchCV(pipe_clf, param_distributions=param, cv=inner, scoring='f1', n_jobs=-1).fit(X_train, y_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = RandomizedSearchCV(pipe_clf, param_distributions=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(X_train, y_train)
final = time.time()
tiempo_bal_acc = final - inicio
inicio = time.time()
y_pred = clf_f1.predict(X_test)
clf_cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_conf_matrix} segundos para confusión matrix.\n")
print(f"El confusion matrix es de: \n{clf_cm}\n")

#--------------------------------------
# Boosting with Hyperparameter Tuning
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with Hyperparameter Tuning
#--------------------------------------

param = {'clf__class_weight': ['balanced', None],
         'clf__max_depth': Integer(low=10, high=20, name='clf__max_depth'),
         'clf__learning_rate': Real(low=0.1, high=1, prior='log-uniform', name='clf__learning_rate')}
inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = RobustScaler()
b_clf = HistGradientBoostingClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = BayesSearchCV(pipe_clf, param, cv=inner, scoring='f1', n_jobs=-1).fit(X_train, y_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = BayesSearchCV(pipe_clf, param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(X_train, y_train)
final = time.time()
tiempo_bal_acc = final - inicio
inicio = time.time()
y_pred = clf_bal_acc.predict(X_test)
clf_cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_conf_matrix} segundos para confusión matrix.\n")
print(f"El confusion matrix es de: \n{clf_cm}\n")

#--------------------------------------
# Boosting with Hyperparameter Tuning
#--------------------------------------

Entre las pruebas y evaluaciones realizadas se ha decidido que el mejor modelo de HistGradientBoosting es el modelo con RandomizedSearchCV. Este modelo ha tenido competencia con GridSearchCV en cuestión a tiempo y métrica, pero al presentar un menor tiempo por algún segundo y tener algún porcentaje mayor que otro Random Search es la mejor opción. Bayes Search no ha sido una opción ya que ha tardado mucho más que los otros y no es que presentase mejoras notables. A continuación, se muestra el modelo con RandomizedSearchCV en un versión de Pipeline mejorada, se tendrá que cambiar 'refit' para conocer los mejores resultados y parámetros:

In [ ]:
#--------------------------------------
# Boosting with Hyperparameter Tuning
#--------------------------------------

param = {'clf__class_weight': ['balanced', None],
         'clf__max_depth': randint(low=10, high=15),
         'clf__learning_rate': uniform( 0.1, 1),
         'clf__random_state': [9]}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = RobustScaler()
scoring = {
    'f1': make_scorer(f1_score),
    'balanced_accuracy': make_scorer(balanced_accuracy_score)
}
clf = HistGradientBoostingClassifier()
pipe_clf = Pipeline([('scaler', scaler), ('clf', clf)])
inicio = time.time()
clf = RandomizedSearchCV(pipe_clf, param, cv=inner, scoring=scoring, refit='f1', n_jobs=-1).fit(X_train, y_train)
y_pred = clf.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo = final - inicio

print(f"\nEl tiempo tardado es de {tiempo} segundos para f1.\n")
print(f"El f1 es de {clf.best_score_} con estimadores {clf.best_estimator_}\n")
print(f"La matriz de confusión es: \n{cm}\n")

#--------------------------------------
# Boosting with Hyperparameter Tuning
#--------------------------------------

##Modelo 2.2: LightGBM

In [ ]:
#--------------------------------------
# Boosting with LightGBM with Hyperparameter Tuning
#--------------------------------------

param = {'clf__n_estimators': [1500, 2000, 2500],
              'clf__learning_rate': [0.1, 1],
              'clf__max_depth': [15, 20, 25],
              'clf__class_weight': ['balanced', None]}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = None
b_clf = LGBMClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring='f1', n_jobs=-1).fit(X_train, y_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(X_train, y_train)
final = time.time()
tiempo_bal_acc = final - inicio
inicio = time.time()
y_pred = clf_f1.predict(X_test)
clf_cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo_conf_matrix = final - inicio


print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_conf_matrix} segundos para confusión matrix.\n")
print(f"El confusion matrix es de: \n{clf_cm}\n")

#--------------------------------------
# Boosting with LightGBM with Hyperparameter Tuning
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with LightGBM with Hyperparameter Tuning
#--------------------------------------

param = {'clf__n_estimators': randint(2000, 3000),
              'clf__learning_rate': loguniform(0.001, 1),
              'clf__max_depth': randint(10, 20),
              'clf__class_weight': ['balanced', None]}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = None
b_clf = LGBMClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = RandomizedSearchCV(pipe_clf, param_distributions=param, cv=inner, scoring='f1', n_jobs=-1).fit(X_train, y_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = RandomizedSearchCV(pipe_clf, param_distributions=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(X_train, y_train)
final = time.time()
tiempo_bal_acc = final - inicio
inicio = time.time()
y_pred = clf_f1.predict(X_test)
clf_cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_conf_matrix} segundos para confusión matrix.\n")
print(f"El confusion matrix es de: \n{clf_cm}\n")

#--------------------------------------
# Boosting with LightGBM with Hyperparameter Tuning
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with LightGBM with Hyperparameter Tuning
#--------------------------------------

param = {'clf__n_estimators': Integer(2000, 3000),
              'clf__learning_rate': Real(0.1, 1, prior='log-uniform'),
              'clf__max_depth': Integer(10, 30)}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = None
b_clf = LGBMClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = BayesSearchCV(pipe_clf, param, cv=inner, scoring='f1', n_jobs=-1).fit(X_train, y_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = BayesSearchCV(pipe_clf, param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(X_train, y_train)
final = time.time()
tiempo_bal_acc = final - inicio
inicio = time.time()
y_pred = clf_bal_acc.predict(X_test)
clf_cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_conf_matrix} segundos para confusión matrix.\n")
print(f"El confusion matrix es de: \n{clf_cm}\n")

#--------------------------------------
# Boosting with LightGBM with Hyperparameter Tuning
#--------------------------------------

Entre las pruebas y evaluaciones realizadas se ha decidido que el mejor modelo de LightGBM es con RandomizedSearchCV. Este modelo ha sido seleccionado por su tiempo ya que tardaba menos en comparación a los demás y aunque los demás presentarán algún cambio en los resultados no han sido significativos como para ser seleccionados. A continuación, se muestra el modelo con RandomizedSearchCV en un versión de Pipeline mejorada, se tendrá que cambiar 'refit' para conocer los mejores resultados y parámetros:

In [ ]:
#--------------------------------------
# Boosting with LightGBM with Hyperparameter Tuning
#--------------------------------------

param = {'clf__n_estimators': randint(2000, 3000),
              'clf__learning_rate': loguniform(0.001, 1),
              'clf__max_depth': randint(10, 20),
              'clf__class_weight': ['balanced', None],
              'clf__random_state': [9]}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = None
scoring = {
    'f1': make_scorer(f1_score),
    'balanced_accuracy': make_scorer(balanced_accuracy_score)
}
clf = LGBMClassifier()
pipe_clf = Pipeline([('scaler', scaler), ('clf', clf)])
inicio = time.time()
clf = RandomizedSearchCV(pipe_clf, param, cv=inner, scoring=scoring, refit='f1', n_jobs=-1).fit(X_train, y_train)
y_pred = clf.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo = final - inicio

print(f"\nEl tiempo tardado es de {tiempo} segundos para f1.\n")
print(f"El f1 es de {clf.best_score_} con estimadores {clf.best_estimator_}\n")
print(f"La matriz de confusión es: \n{cm}\n")

#--------------------------------------
# Boosting with LightGBM with Hyperparameter Tuning
#--------------------------------------

#Modelos con atributos de tipo filtro

Utilizaremos SelectKBest sobre 10 para aplicar filtros a los atributos. Esta aplicación unicamente se aplica sobre los de entrenamiento y no a todos los datos.

##Datos con atributos de tipo filtro

In [ ]:
#------------------------------------------
#Conjuntos para f_classif
selector_f_classif = SelectKBest(f_classif, k=10)
Z_train = selector_f_classif.fit_transform(X_train, y_train)
Z_test = selector_f_classif.fit_transform(X_test, y_test)

#------------------------------------------
#Conjuntos para mutual_info_classif
selector_mutual_info_classif = SelectKBest(mutual_info_classif, k=10)
W_train = selector_mutual_info_classif.fit_transform(X_train, y_train)
W_test = selector_mutual_info_classif.fit_transform(X_test, y_test)

#------------------------------------------
#Conjuntos para chi2
selector_chi2_classif = SelectKBest(chi2, k=10)
V_train = selector_chi2_classif.fit_transform(X_train, y_train)
V_test = selector_chi2_classif.fit_transform(X_test, y_test)

##Modelos con atributos de tipo filtro f_classif

In [ ]:
#--------------------------------------
# Boosting with Hyperparameter Tuning with filter
#--------------------------------------

param = { 'clf__class_weight': ['balanced', None],
         'clf__max_depth': [10, 15, 20],
         'clf__learning_rate': [10, 1, 0.1]}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = None
b_clf = HistGradientBoostingClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring='f1', n_jobs=-1).fit(Z_train, y_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(Z_train, y_train)
final = time.time()
tiempo_bal_acc = final - inicio
inicio = time.time()
y_pred = clf_f1.predict(Z_test)
clf_cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_conf_matrix} segundos para confusión matrix.\n")
print(f"El confusion matrix es de: \n{clf_cm}\n")

#--------------------------------------
# Boosting with Hyperparameter Tuning with filter
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with Hyperparameter Tuning with filter
#--------------------------------------

param = {'clf__class_weight': ['balanced', None],
         'clf__max_depth': randint(low=10, high=20),
         'clf__learning_rate': uniform(0.01, 1)}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = StandardScaler()
b_clf = HistGradientBoostingClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = RandomizedSearchCV(pipe_clf, param_distributions=param, cv=inner, scoring='f1', n_jobs=-1).fit(Z_train, y_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = RandomizedSearchCV(pipe_clf, param_distributions=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(Z_train, y_train)
final = time.time()
tiempo_bal_acc = final - inicio
inicio = time.time()
y_pred = clf_f1.predict(Z_test)
clf_cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_conf_matrix} segundos para confusión matrix.\n")
print(f"El confusion matrix es de: \n{clf_cm}\n")

#--------------------------------------
# Boosting with Hyperparameter Tuning with filter
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with Hyperparameter Tuning with filter
#--------------------------------------

param = {'clf__class_weight': ['balanced', None],
         'clf__max_depth': Integer(low=20, high=70, name='clf__max_depth'),
         'clf__learning_rate': Real(low=0.01, high=10, prior='log-uniform', name='clf__learning_rate')}
inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = RobustScaler()
b_clf = HistGradientBoostingClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = BayesSearchCV(pipe_clf, param, cv=inner, scoring='f1', n_jobs=-1).fit(Z_train, y_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = BayesSearchCV(pipe_clf, param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(Z_train, y_train)
final = time.time()
tiempo_bal_acc = final - inicio
inicio = time.time()
y_pred = clf_bal_acc.predict(Z_test)
clf_cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_conf_matrix} segundos para confusión matrix.\n")
print(f"El confusion matrix es de: \n{clf_cm}\n")

#--------------------------------------
# Boosting with Hyperparameter Tuning with filter
#--------------------------------------

Entre las pruebas y evaluaciones realizadas se ha decidido que el mejor modelo de HistGradientBoosting con filtro de f_classif es con GridSearchCV. Como pasará en todos los modelos Bayes Search tiene tiempos muy elevados y es por eso que se descarta. En cuanto a Random Search, presenta un tiempo y resultados muy similares a Random Search por lo que la decisión ha estado con nostros y la matriz de confusión. A continuación, se muestra el modelo con RandomizedSearchCV en un versión de Pipeline mejorada, se tendrá que cambiar 'refit' para conocer los mejores resultados y parámetros: 

In [ ]:
#--------------------------------------
# Boosting with LightGBM with Hyperparameter Tuning
#--------------------------------------

param = {'clf__class_weight': ['balanced', None],
         'clf__max_depth': [10, 15, 20],
         'clf__learning_rate': [10, 1, 0.1],
         'clf__random_state': [9]}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = None
scoring = {
    'f1': make_scorer(f1_score),
    'balanced_accuracy': make_scorer(balanced_accuracy_score)
}
clf = HistGradientBoostingClassifier()
pipe_clf = Pipeline([('scaler', scaler), ('clf', clf)])
inicio = time.time()
clf = GridSearchCV(pipe_clf, param, cv=inner, scoring=scoring, refit='f1', n_jobs=-1).fit(Z_train, y_train)
y_pred = clf.predict(Z_test)
cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo = final - inicio

print(f"\nEl tiempo tardado es de {tiempo} segundos para f1.\n")
print(f"El f1 es de {clf.best_score_} con estimadores {clf.best_estimator_}\n")
print(f"La matriz de confusión es: \n{cm}\n")

#--------------------------------------
# Boosting with LightGBM with Hyperparameter Tuning
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with LightGBM with filter
#--------------------------------------

param = {'clf__n_estimators': [2000, 3000],
              'clf__learning_rate': [0.1, 1, 10],
              'clf__max_depth': [10, 15, 20],
              'clf__class_weight': ['balanced', None]}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = None
b_clf = LGBMClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring='f1', n_jobs=-1).fit(Z_train, y_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(Z_train, y_train)
final = time.time()
tiempo_bal_acc = final - inicio
inicio = time.time()
y_pred = clf_f1.predict(Z_test)
clf_cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo_conf_matrix = final - inicio


print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_conf_matrix} segundos para confusión matrix.\n")
print(f"El confusion matrix es de: \n{clf_cm}\n")

#--------------------------------------
# Boosting with LightGBM with filter
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with LightGBM with filter
#--------------------------------------

param = {'clf__n_estimators': randint(1000, 3000),
              'clf__learning_rate': loguniform(0.01, 1),
              'clf__max_depth': randint(10, 20),
              'clf__class_weight': ['balanced', None]}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = StandardScaler()
b_clf = LGBMClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = RandomizedSearchCV(pipe_clf, param_distributions=param, cv=inner, scoring='f1', n_jobs=-1).fit(Z_train, y_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = RandomizedSearchCV(pipe_clf, param_distributions=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(Z_train, y_train)
final = time.time()
tiempo_bal_acc = final - inicio
inicio = time.time()
y_pred = clf_f1.predict(Z_test)
clf_cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_conf_matrix} segundos para confusión matrix.\n")
print(f"El confusion matrix es de: \n{clf_cm}\n")

#--------------------------------------
# Boosting with LightGBM with filter
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with LightGBM with filter
#--------------------------------------

param = {'clf__n_estimators': Integer(1000, 3000),
              'clf__learning_rate': Real(0.1, 1, prior='log-uniform'),
              'clf__max_depth': Integer(10, 20)}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = RobustScaler()
b_clf = LGBMClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = BayesSearchCV(pipe_clf, param, cv=inner, scoring='f1', n_jobs=-1).fit(Z_train, y_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = BayesSearchCV(pipe_clf, param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(Z_train, y_train)
final = time.time()
tiempo_bal_acc = final - inicio
inicio = time.time()
y_pred = clf_bal_acc.predict(Z_test)
clf_cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_conf_matrix} segundos para confusión matrix.\n")
print(f"El confusion matrix es de: \n{clf_cm}\n")

#--------------------------------------
# Boosting with LightGBM with filter
#--------------------------------------

Entre las pruebas y evaluaciones realizadas se ha decidido que el mejor modelo de LightGBM con filtro de f_classif es con Grid Search. Bayes Search es modelo que descartamos por tiempo. En cuanto a Random Search, este presenta un tiempo mucho menor y resultados muy similares a Grid Search, pero ha influenciado en la decisión ha sido la matriz de confusión que a diferencia de otros casos anteriores hay un diferencia notable en la que Grid Search es mejor. A continuación, se muestra el modelo con GridSearchCV en un versión de Pipeline mejorada, se tendrá que cambiar 'refit' para conocer los mejores resultados y parámetros:

In [ ]:
#--------------------------------------
# Boosting with LightGBM with filter
#--------------------------------------

param = {'clf__n_estimators': [2000, 3000],
              'clf__learning_rate': [0.1, 1, 10],
              'clf__max_depth': [10, 15, 20],
              'clf__class_weight': ['balanced', None],
              'clf__random_state': [9]}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = None
scoring = {
    'f1': make_scorer(f1_score),
    'balanced_accuracy': make_scorer(balanced_accuracy_score)
}
clf = LGBMClassifier()
pipe_clf = Pipeline([('scaler', scaler), ('clf', clf)])
inicio = time.time()
clf = GridSearchCV(pipe_clf, param, cv=inner, scoring=scoring, refit='f1', n_jobs=-1).fit(Z_train, y_train)
y_pred = clf.predict(Z_test)
cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo = final - inicio

print(f"\nEl tiempo tardado es de {tiempo} segundos para f1.\n")
print(f"El f1 es de {clf.best_score_} con estimadores {clf.best_estimator_}\n")
print(f"La matriz de confusión es: \n{cm}\n")

#--------------------------------------
# Boosting with LightGBM with filter
#--------------------------------------

##Modelos con atributos de tipo filtro mutual_info_classif

In [ ]:
#--------------------------------------
# Boosting with Hyperparameter Tuning with filter
#--------------------------------------

param = { 'clf__class_weight': ['balanced', None],
         'clf__max_depth': [10, 15, 20],
         'clf__learning_rate': [10, 1, 0.1]}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = None
b_clf = HistGradientBoostingClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring='f1', n_jobs=-1).fit(W_train, y_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(W_train, y_train)
final = time.time()
tiempo_bal_acc = final - inicio
inicio = time.time()
y_pred = clf_f1.predict(W_test)
clf_cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_conf_matrix} segundos para confusión matrix.\n")
print(f"El confusion matrix es de: \n{clf_cm}\n")

#--------------------------------------
# Boosting with Hyperparameter Tuning with filter
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with Hyperparameter Tuning with filter
#--------------------------------------

param = {'clf__class_weight': ['balanced', None],
         'clf__max_depth': randint(low=10, high=20),
         'clf__learning_rate': uniform(0.01, 1)}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = RobustScaler()
b_clf = HistGradientBoostingClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = RandomizedSearchCV(pipe_clf, param_distributions=param, cv=inner, scoring='f1', n_jobs=-1).fit(W_train, y_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = RandomizedSearchCV(pipe_clf, param_distributions=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(W_train, y_train)
final = time.time()
tiempo_bal_acc = final - inicio
inicio = time.time()
y_pred = clf_f1.predict(W_test)
clf_cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_conf_matrix} segundos para confusión matrix.\n")
print(f"El confusion matrix es de: \n{clf_cm}\n")

#--------------------------------------
# Boosting with Hyperparameter Tuning with filter
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with Hyperparameter Tuning with filter
#--------------------------------------

param = {'clf__class_weight': ['balanced', None],
         'clf__max_depth': Integer(low=10, high=20, name='clf__max_depth'),
         'clf__learning_rate': Real(low=0.01, high=1, prior='log-uniform', name='clf__learning_rate')}
inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = None
b_clf = HistGradientBoostingClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = BayesSearchCV(pipe_clf, param, cv=inner, scoring='f1', n_jobs=-1).fit(W_train, y_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = BayesSearchCV(pipe_clf, param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(W_train, y_train)
final = time.time()
tiempo_bal_acc = final - inicio
inicio = time.time()
y_pred = clf_bal_acc.predict(W_test)
clf_cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_conf_matrix} segundos para confusión matrix.\n")
print(f"El confusion matrix es de: \n{clf_cm}\n")

#--------------------------------------
# Boosting with Hyperparameter Tuning with filter
#--------------------------------------

Entre las pruebas y evaluaciones realizadas se ha decidido que el mejor modelo de HistGradientBoosting con filtro de mutual_info_classf es con RandomSeacrhCV. Bayes Search descartado por sus enormes tiempos. Grid Search y Random Search presentan tiempos y resultados muy similares, pero la matriz de confusión ha sido la que ha influenciado en la decisión. A continuación, se muestra el modelo con RandomSeacrhCV en un versión de Pipeline mejorada, se tendrá que cambiar 'refit' para conocer los mejores resultados y parámetros:

In [ ]:
#--------------------------------------
# Boosting with Hyperparameter Tuning with filter
#--------------------------------------

param = { 'clf__class_weight': ['balanced', None],
         'clf__max_depth': randint(low=10, high=20),
         'clf__learning_rate': uniform(0.01, 1),
         'clf__random_state': [9]}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = RobustScaler()
scoring = {
    'f1': make_scorer(f1_score),
    'balanced_accuracy': make_scorer(balanced_accuracy_score)
}
clf = HistGradientBoostingClassifier()
pipe_clf = Pipeline([('scaler', scaler), ('clf', clf)])
inicio = time.time()
clf = RandomizedSearchCV(pipe_clf, param, cv=inner, scoring=scoring, refit='f1', n_jobs=-1).fit(W_train, y_train)
y_pred = clf.predict(W_test)
cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo = final - inicio

print(f"\nEl tiempo tardado es de {tiempo} segundos para f1.\n")
print(f"El f1 es de {clf.best_score_} con estimadores {clf.best_estimator_}\n")
print(f"La matriz de confusión es: \n{cm}\n")

#--------------------------------------
# Boosting with Hyperparameter Tuning with filter
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with LightGBM with filter
#--------------------------------------

param = {'clf__n_estimators': [1000, 1500, 2000],
              'clf__learning_rate': [0.01, 0.1, 1],
              'clf__max_depth': [10, 15, 20],
              'clf__class_weight': ['balanced', None]}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = None
b_clf = LGBMClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring='f1', n_jobs=-1).fit(W_train, y_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(W_train, y_train)
final = time.time()
tiempo_bal_acc = final - inicio
inicio = time.time()
y_pred = clf_f1.predict(W_test)
clf_cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo_conf_matrix = final - inicio


print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_conf_matrix} segundos para confusión matrix.\n")
print(f"El confusion matrix es de: \n{clf_cm}\n")

#--------------------------------------
# Boosting with LightGBM with filter
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with LightGBM with filter
#--------------------------------------

param = {'clf__n_estimators': randint(1000, 3000),
              'clf__learning_rate': loguniform(0.01, 1),
              'clf__max_depth': randint(10, 20),
              'clf__class_weight': ['balanced', None]}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = StandardScaler()
b_clf = LGBMClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = RandomizedSearchCV(pipe_clf, param_distributions=param, cv=inner, scoring='f1', n_jobs=-1).fit(W_train, y_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = RandomizedSearchCV(pipe_clf, param_distributions=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(W_train, y_train)
final = time.time()
tiempo_bal_acc = final - inicio
inicio = time.time()
y_pred = clf_f1.predict(W_test)
clf_cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_conf_matrix} segundos para confusión matrix.\n")
print(f"El confusion matrix es de: \n{clf_cm}\n")

#--------------------------------------
# Boosting with LightGBM with filter
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with LightGBM with filter
#--------------------------------------

param = {'clf__n_estimators': Integer(500, 2000),
              'clf__learning_rate': Real(0.01, 1, prior='log-uniform'),
              'clf__max_depth': Integer(10, 20)}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = StandardScaler()
b_clf = LGBMClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = BayesSearchCV(pipe_clf, param, cv=inner, scoring='f1', n_jobs=-1).fit(W_train, y_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = BayesSearchCV(pipe_clf, param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(W_train, y_train)
final = time.time()
tiempo_bal_acc = final - inicio
inicio = time.time()
y_pred = clf_bal_acc.predict(W_test)
clf_cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_conf_matrix} segundos para confusión matrix.\n")
print(f"El confusion matrix es de: \n{clf_cm}\n")

#--------------------------------------
# Boosting with LightGBM with filter
#--------------------------------------

Entre las pruebas y evaluaciones realizadas se ha decidido que el mejor modelo de LightGBM con filtro de mutual_info_classf es con RandomSearchCV. Bayes Search y Grid Search presentan tiempos muy altos por lo que se descartan inmediatamente ya que ninguno presenta mucha mejora en relación a Random Search. A continuación, se muestra el modelo con RandomizedSearchCV en un versión de Pipeline mejorada, se tendrá que cambiar 'refit' para conocer los mejores resultados y parámetros:

In [ ]:
#--------------------------------------
# Boosting with LightGBM with filter
#--------------------------------------

param = {'clf__n_estimators': randint(1000, 3000),
              'clf__learning_rate': loguniform(0.01, 1),
              'clf__max_depth': randint(10, 20),
              'clf__class_weight': ['balanced', None],
              'clf__random_state': [9]}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = StandardScaler()
scoring = {
    'f1': make_scorer(f1_score),
    'balanced_accuracy': make_scorer(balanced_accuracy_score)
}
clf = LGBMClassifier()
pipe_clf = Pipeline([('scaler', scaler), ('clf', clf)])
inicio = time.time()
clf = RandomizedSearchCV(pipe_clf, param, cv=inner, scoring=scoring, refit='f1', n_jobs=-1).fit(W_train, y_train)
y_pred = clf.predict(W_test)
cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo = final - inicio

print(f"\nEl tiempo tardado es de {tiempo} segundos para f1.\n")
print(f"El f1 es de {clf.best_score_} con estimadores {clf.best_estimator_}\n")
print(f"La matriz de confusión es: \n{cm}\n")

#--------------------------------------
# Boosting with LightGBM with filter
#--------------------------------------

##Modelos con atributos de tipo filtro chi2

In [ ]:
#--------------------------------------
# Boosting with Hyperparameter Tuning with filter
#--------------------------------------

param = { 'clf__class_weight': ['balanced', None],
         'clf__max_depth': [10, 15, 20],
         'clf__learning_rate': [10, 1, 0.1]}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = None
b_clf = HistGradientBoostingClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring='f1', n_jobs=-1).fit(V_train, y_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(V_train, y_train)
final = time.time()
tiempo_bal_acc = final - inicio
inicio = time.time()
y_pred = clf_f1.predict(V_test)
clf_cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_conf_matrix} segundos para confusión matrix.\n")
print(f"El confusion matrix es de: \n{clf_cm}\n")

#--------------------------------------
# Boosting with Hyperparameter Tuning with filter
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with Hyperparameter Tuning with filter
#--------------------------------------

param = {'clf__class_weight': ['balanced', None],
         'clf__max_depth': randint(low=10, high=20),
         'clf__learning_rate': uniform(0.1, 1)}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = StandardScaler()
b_clf = HistGradientBoostingClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = RandomizedSearchCV(pipe_clf, param_distributions=param, cv=inner, scoring='f1', n_jobs=-1).fit(V_train, y_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = RandomizedSearchCV(pipe_clf, param_distributions=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(V_train, y_train)
final = time.time()
tiempo_bal_acc = final - inicio
inicio = time.time()
y_pred = clf_f1.predict(V_test)
clf_cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_conf_matrix} segundos para confusión matrix.\n")
print(f"El confusion matrix es de: \n{clf_cm}\n")

#--------------------------------------
# Boosting with Hyperparameter Tuning with filter
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with Hyperparameter Tuning with filter
#--------------------------------------

param = {'clf__class_weight': ['balanced', None],
         'clf__max_depth': Integer(low=10, high=20, name='clf__max_depth'),
         'clf__learning_rate': Real(low=0.1, high=1, prior='log-uniform', name='clf__learning_rate')}
inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = None
b_clf = HistGradientBoostingClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = BayesSearchCV(pipe_clf, param, cv=inner, scoring='f1', n_jobs=-1).fit(V_train, y_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = BayesSearchCV(pipe_clf, param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(V_train, y_train)
final = time.time()
tiempo_bal_acc = final - inicio
inicio = time.time()
y_pred = clf_bal_acc.predict(V_test)
clf_cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_conf_matrix} segundos para confusión matrix.\n")
print(f"El confusion matrix es de: \n{clf_cm}\n")

#--------------------------------------
# Boosting with Hyperparameter Tuning with filter
#--------------------------------------

Entre las pruebas y evaluaciones realizadas se ha decidido que el mejor modelo de HistGradientBoosting con filtro de chi2 es con RandomSearchCV. Bayes como se ha mencionado en muchas explicaciones no se toma en cuenta por sus tiempos. En cuanto a Grid Search y Random Search, presentan tiempos y resultados muy similares, pero la matriz de confusión ha sido la que ha influenciado en la decisión. A continuación, se muestra el modelo con RandomizedSearchCV en un versión de Pipeline mejorada, se tendrá que cambiar 'refit' para conocer los mejores resultados y parámetros:

In [ ]:
#--------------------------------------
# Boosting with Hyperparameter Tuning with filter
#--------------------------------------

param = {'clf__class_weight': ['balanced', None],
         'clf__max_depth': randint(low=10, high=20),
         'clf__learning_rate': uniform(0.1, 1),
         'clf__random_state': [9]}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = StandardScaler()
scoring = {
    'f1': make_scorer(f1_score),
    'balanced_accuracy': make_scorer(balanced_accuracy_score)
}
clf = HistGradientBoostingClassifier()
pipe_clf = Pipeline([('scaler', scaler), ('clf', clf)])
inicio = time.time()
clf = RandomizedSearchCV(pipe_clf, param, cv=inner, scoring=scoring, refit='f1', n_jobs=-1).fit(V_train, y_train)
y_pred = clf.predict(V_test)
cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo = final - inicio

print(f"\nEl tiempo tardado es de {tiempo} segundos para f1.\n")
print(f"El f1 es de {clf.best_score_} con estimadores {clf.best_estimator_}\n")
print(f"La matriz de confusión es: \n{cm}\n")

#--------------------------------------
# Boosting with Hyperparameter Tuning with filter
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with LightGBM with filter
#--------------------------------------

param = {'clf__n_estimators': [250, 500, 1000],
              'clf__learning_rate': [0.1, 1, 10],
              'clf__max_depth': [10, 15, 20],
              'clf__class_weight': ['balanced', None]}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = None
b_clf = LGBMClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring='f1', n_jobs=-1).fit(V_train, y_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = GridSearchCV(pipe_clf, param_grid=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(V_train, y_train)
final = time.time()
tiempo_bal_acc = final - inicio
inicio = time.time()
y_pred = clf_f1.predict(V_test)
clf_cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo_conf_matrix = final - inicio


print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_conf_matrix} segundos para confusión matrix.\n")
print(f"El confusion matrix es de: \n{clf_cm}\n")

#--------------------------------------
# Boosting with LightGBM with filter
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with LightGBM with filter
#--------------------------------------

param = {'clf__n_estimators': randint(1000, 3000),
              'clf__learning_rate': loguniform(0.1, 1),
              'clf__max_depth': randint(10, 20),
              'clf__class_weight': ['balanced', None]}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = RobustScaler()
b_clf = LGBMClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = RandomizedSearchCV(pipe_clf, param_distributions=param, cv=inner, scoring='f1', n_jobs=-1).fit(V_train, y_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = RandomizedSearchCV(pipe_clf, param_distributions=param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(V_train, y_train)
final = time.time()
tiempo_bal_acc = final - inicio
inicio = time.time()
y_pred = clf_f1.predict(V_test)
clf_cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_conf_matrix} segundos para confusión matrix.\n")
print(f"El confusion matrix es de: \n{clf_cm}\n")

#--------------------------------------
# Boosting with LightGBM with filter
#--------------------------------------

In [ ]:
#--------------------------------------
# Boosting with LightGBM with filter
#--------------------------------------

param = {'clf__n_estimators': Integer(1000, 3000),
              'clf__learning_rate': Real(0.01, 1, prior='log-uniform'),
              'clf__max_depth': Integer(10, 20)}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = RobustScaler()
b_clf = LGBMClassifier(random_state=9)
pipe_clf = Pipeline([('scaler', scaler), ('clf', b_clf)])
inicio = time.time()
clf_f1 = BayesSearchCV(pipe_clf, param, cv=inner, scoring='f1', n_jobs=-1).fit(V_train, y_train)
final = time.time()
tiempo_f1 = final - inicio
inicio = time.time()
clf_bal_acc = BayesSearchCV(pipe_clf, param, cv=inner, scoring= 'balanced_accuracy', n_jobs=-1).fit(V_train, y_train)
final = time.time()
tiempo_bal_acc = final - inicio
inicio = time.time()
y_pred = clf_bal_acc.predict(V_test)
clf_cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo_conf_matrix = final - inicio

print(f"El tiempo tardado es de {tiempo_f1} segundos para f1.\n")
print(f"El f1 es de {clf_f1.best_score_} con estimadores {clf_f1.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_bal_acc} segundos para balanced accuracy.\n")
print(f"El balanced accuracy es de {clf_bal_acc.best_score_} con estimadores {clf_bal_acc.best_estimator_}\n")
print(f"El tiempo tardado es de {tiempo_conf_matrix} segundos para confusión matrix.\n")
print(f"El confusion matrix es de: \n{clf_cm}\n")

#--------------------------------------
# Boosting with LightGBM with filter
#--------------------------------------

Entre las pruebas y evaluaciones realizadas se ha decidido que el mejor modelo de LightGBM con filtro de chi2 es con RandomSearchCV. Bayes Search presenta tiempos muy elevados. En cuanto a Grid Search y Random Search, presentan tiempos y resultados muy similares, pero el tiempo en este caso ha sido la que ha influenciado en la decisión. A continuación, se muestra el modelo con RandomizedSearchCV en un versión de Pipeline mejorada, se tendrá que cambiar 'refit' para conocer los mejores resultados y parámetros:

In [ ]:
#--------------------------------------
# Boosting with LightGBM with filter
#--------------------------------------

param = {'clf__n_estimators': randint(1000, 3000),
              'clf__learning_rate': loguniform(0.1, 1),
              'clf__max_depth': randint(10, 20),
              'clf__class_weight': ['balanced', None],
              'clf__random_state': [9]}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = RobustScaler()
scoring = {
    'f1': make_scorer(f1_score),
    'balanced_accuracy': make_scorer(balanced_accuracy_score)
}
clf = LGBMClassifier()
pipe_clf = Pipeline([('scaler', scaler), ('clf', clf)])
inicio = time.time()
clf = RandomizedSearchCV(pipe_clf, param, cv=inner, scoring=scoring, refit='f1', n_jobs=-1).fit(V_train, y_train)
y_pred = clf.predict(V_test)
cm = confusion_matrix(y_test, y_pred)
final = time.time()
tiempo = final - inicio

print(f"\nEl tiempo tardado es de {tiempo} segundos para f1.\n")
print(f"El f1 es de {clf.best_score_} con estimadores {clf.best_estimator_}\n")
print(f"La matriz de confusión es: \n{cm}\n")

#--------------------------------------
# Boosting with LightGBM with filter
#--------------------------------------

#Conclusiones y comparación de modelos

Por las conclusiones sacadas de cada tipo de modelo con sus características la regresión logística es nefasta en relación a los demás modelos por lo que no será el modelo final. En el caso de boosting se han comprobado y comparado los resultados de HistGradientBoosting con y sin filtro y por lo que se ha visto el mejor es sin filtros, esto es porque algunos o la mayoría presentaban mejores tiempos pero a la hora de comparar resultados de las métricas o se parecían bastante pero la matriz era mucho peor o presentaban peores resultados es por eso que el vencedor este modelo sin filtro. De la misma manera a ocurrido al comparar las distintas versiones para el modelo de LightGBM y se concluye que es mejor sin filtros. Ahora la desición entre estos dos es complicada ya que ambos presentan resultados similares en todas las métricas por lo que nos guiaremos por el tiempo tardado. En este cosa y bajo este criterio será HistGradientBoosting con RandomSearchCV el modelo final.  

Finalmente, queremos mencionar que hemos tenido problemas para poder añadir el preproceso de datos en las pipelines de los modelos por lo que he lo hemos hecho al principio sobre todo el conjunto de datos. Sabemos que los hiperparámetros escogidos no sean los mejores para dichos modelos pero tras una serie de evaluaciones no hemos decantado por los que se muestran en el notebook. 

#Modelo final

In [ ]:
#--------------------------------------
# Final Model with HistGradientBoostingClassifier RandomizedSearchCV
#--------------------------------------

param = {'clf__class_weight': ['balanced', None],
         'clf__max_depth': randint(low=10, high=15),
         'clf__learning_rate': uniform( 0.1, 1),
         'clf__random_state': [9]}

inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)
scaler = RobustScaler()
scoring = {
    'f1': make_scorer(f1_score),
    'balanced_accuracy': make_scorer(balanced_accuracy_score)
}
clf = HistGradientBoostingClassifier()
pipe_clf = Pipeline([('scaler', scaler), ('clf', clf)])
clf = RandomizedSearchCV(pipe_clf, param, cv=inner, scoring=scoring, refit='f1', n_jobs=-1).fit(X_train, y_train)

#--------------------------------------
# Passing the model to pickle
#--------------------------------------
with open('modelo_final.pkl', 'wb') as file:
    pickle.dump(clf, file)